<a href="https://colab.research.google.com/github/randy-ar/gcolab/blob/main/Preprocessing_data_EPA_Chemical_Data_Reporting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import gdown
import nltk
import matplotlib.pyplot as plt

# Read Data CSV

In [5]:
table_consumer_and_use_information = "1dGgcbPnVmOeAP03MMG_jf7FwfQEoTa2P"
table_industrial_processing_and_use_information = "1uS0ucuC24KjENVmqssHiQeL05n4j-LNd"
table_nationally_aggregated_production_volumes = "1bTz5YRHcw--kzIGEbUQSC8taqCrAxGaG"

download_url = "https://docs.google.com/uc?export=download&id="

In [6]:
gdown.download(download_url+table_consumer_and_use_information, 'table_consumer_and_use_information.csv', quiet=False)
gdown.download(download_url+table_industrial_processing_and_use_information, 'table_industrial_processing_and_use_information.csv', quiet=False)
gdown.download(download_url+table_nationally_aggregated_production_volumes, 'table_nationally_aggregated_production_volumes.csv', quiet=False)

Downloading...
From: https://docs.google.com/uc?export=download&id=1dGgcbPnVmOeAP03MMG_jf7FwfQEoTa2P
To: /content/table_consumer_and_use_information.csv
100%|██████████| 37.0M/37.0M [00:00<00:00, 48.7MB/s]
Downloading...
From: https://docs.google.com/uc?export=download&id=1uS0ucuC24KjENVmqssHiQeL05n4j-LNd
To: /content/table_industrial_processing_and_use_information.csv
100%|██████████| 48.2M/48.2M [00:00<00:00, 55.5MB/s]
Downloading...
From: https://docs.google.com/uc?export=download&id=1bTz5YRHcw--kzIGEbUQSC8taqCrAxGaG
To: /content/table_nationally_aggregated_production_volumes.csv
100%|██████████| 1.32M/1.32M [00:00<00:00, 20.8MB/s]


'table_nationally_aggregated_production_volumes.csv'

In [8]:
df_consumer = pd.read_csv('table_consumer_and_use_information.csv')
df_industrial = pd.read_csv('table_industrial_processing_and_use_information.csv')
df_nationally = pd.read_csv('table_nationally_aggregated_production_volumes.csv')

<ipython-input-8-3d8892f2bb02>:1: DtypeWarning: Columns (69) have mixed types. Specify dtype option on import or set low_memory=False.
  df_consumer = pd.read_csv('table_consumer_and_use_information.csv')
<ipython-input-8-3d8892f2bb02>:2: DtypeWarning: Columns (71) have mixed types. Specify dtype option on import or set low_memory=False.
  df_industrial = pd.read_csv('table_industrial_processing_and_use_information.csv')


In [9]:
df_consumer.head()

,CHEMICAL REPORT ID,CHEMICAL NAME,CHEMICAL ID,CHEMICAL ID W/O DASHES,CHEMICAL ID TYPE,DOMESTIC PARENT COMPANY NAME,DOMESTIC PC ADDRESS LINE1,DOMESTIC PC ADDRESS LINE2,DOMESTIC PC CITY,DOMESTIC PC COUNTY / PARISH,...,JOINT FC CODE,JOINT FUNCTION CATEGORY,JOINT FUNCT CAT OTHER DESC,CONS OR COMM USE,USED IN PROD FOR CHILDREN,C / C PV PCT,C / C MAX CONC CODE,C / C MAXIMUM CONCENTRATION,COMM WORKERS CODE,COMMERCIAL WORKERS
0,711448.0,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849.0,Accession Number,CBI,CBI,CBI,CBI,CBI,...,NaN,NaN,NaN,Commercial,No,100,NKRA,Not Known or Reasonably Ascertainable,NKRA,Not Known or Reasonably Ascertainable
1,728903.0,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849.0,Accession Number,CBI,CBI,CBI,CBI,CBI,...,NaN,NaN,NaN,Both,No,100,M2,1% – < 30%,W8,"10,000+"
2,702175.0,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849.0,Accession Number,CBI,CBI,CBI,CBI,CBI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,704950.0,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849.0,Accession Number,IDEMITSU LUBRICANTS AMERICA,701 PORT RD,NaN,JEFFERSONVILLE,Clark,...,NaN,NaN,NaN,Both,No,93,M2,1% – < 30%,NKRA,Not Known or Reasonably Ascertainable
4,735410.0,"(Polyisobutenyl)dihydro-2,5-furandione reactio...",24103,24103.0,Accession Number,CBI,CBI,CBI,CBI,CBI,...,NaN,NaN,NaN,Commercial,No,100,NKRA,Not Known or Reasonably Ascertainable,NKRA,Not Known or Reasonably Ascertainable


In [10]:
df_industrial.head()

,CHEMICAL REPORT ID,CHEMICAL NAME,CHEMICAL ID,CHEMICAL ID W/O DASHES,CHEMICAL ID TYPE,DOMESTIC PARENT COMPANY NAME,DOMESTIC PC ADDRESS LINE1,DOMESTIC PC ADDRESS LINE2,DOMESTIC PC CITY,DOMESTIC PC COUNTY / PARISH,...,INDUSTRIAL FUNCTION CATEGORY,IND FUNCT CAT OTHER DESC,JOINT FC CODE,JOINT FUNCTION CATEGORY,JOINT FUNCT CAT OTHER DESC,IND PV PCT,IND SITES CODE,INDUSTRIAL SITES,IND WORKERS CODE,INDUSTRIAL WORKERS
0,711448.0,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849.0,Accession Number,CBI,CBI,CBI,CBI,CBI,...,Lubricants and lubricant additives,NaN,NaN,NaN,NaN,100,NKRA,Not Known or Reasonably Ascertainable,NKRA,Not Known or Reasonably Ascertainable
1,728903.0,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849.0,Accession Number,CBI,CBI,CBI,CBI,CBI,...,Lubricants and lubricant additives,NaN,NaN,NaN,NaN,100,S4,100 – 249,W7,"1,000 – 9,999"
2,702175.0,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849.0,Accession Number,CBI,CBI,CBI,CBI,CBI,...,Corrosion inhibitor,NaN,NaN,NaN,NaN,CBI,CBI,CBI,W1,< 10
3,702175.0,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849.0,Accession Number,CBI,CBI,CBI,CBI,CBI,...,Processing aids not otherwise specified,NaN,NaN,NaN,NaN,CBI,CBI,CBI,W3,25 – 49
4,702175.0,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849.0,Accession Number,CBI,CBI,CBI,CBI,CBI,...,Corrosion inhibitor,NaN,NaN,NaN,NaN,CBI,CBI,CBI,W1,< 10


In [11]:
df_nationally.head()

,CHEMICAL NAME,CHEMICAL ID,CHEMICAL ID W/O DASHES,CHEMICAL ID TYPE,2019 NATIONALLY AGGREGATED PV,2018 NATIONALLY AGGREGATED PV,2017 NATIONALLY AGGREGATED PV,2016 NATIONALLY AGGREGATED PV
0,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849,Accession Number,"100,000 – <500,000","1,000,000 – <10,000,000","500,000 – <1,000,000","1,000,000 – <20,000,000"
1,"(Polyisobutenyl)dihydro-2,5-furandione reactio...",24103,24103,Accession Number,"<1,000,000","1,000,000 – <20,000,000","1,000,000 – <20,000,000","1,000,000 – <20,000,000"
2,"(Polyisobutenyl)dihydro-2,5-furandione reactio...",12556,12556,Accession Number,"20,000,000 – <100,000,000","1,000,000 – <20,000,000","20,000,000 – <100,000,000","1,000,000 – <20,000,000"
3,"(Polyisobutenyl)dihydro-2,5-furandione, polyhy...",22629,22629,Accession Number,"<1,000,000","<1,000,000","<1,000,000","<1,000,000"
4,"(Polyisobutenyl)dihydro-2,5-furandione, reacti...",18805,18805,Accession Number,"1,000,000 – <20,000,000","<1,000,000","<1,000,000","<1,000,000"


# Preprocessing Data

In [12]:
df_consumer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56032 entries, 0 to 56031
Data columns (total 77 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   CHEMICAL REPORT ID                      56000 non-null  float64
 1   CHEMICAL NAME                           56000 non-null  object 
 2   CHEMICAL ID                             56000 non-null  object 
 3   CHEMICAL ID W/O DASHES                  56000 non-null  float64
 4   CHEMICAL ID TYPE                        56000 non-null  object 
 5   DOMESTIC PARENT COMPANY NAME            56000 non-null  object 
 6   DOMESTIC PC ADDRESS LINE1               56000 non-null  object 
 7   DOMESTIC PC ADDRESS LINE2               15189 non-null  object 
 8   DOMESTIC PC CITY                        56000 non-null  object 
 9   DOMESTIC PC COUNTY / PARISH             55998 non-null  object 
 10  DOMESTIC PC STATE                       56000 non-null  ob